# PV-Production function 

In [1]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
import pvlib
from pvlib import pvsystem

In [2]:
def prod(location,irrad_file, met_file, start, end,mod, inver ,module,inverter, surface_azimuth, surface_tilt,modules_per_string,
        strings_per_inverter):
    cop = location
    # Open Irra data
    df=pd.read_csv(irrad_file, header=42,sep=";" )
    # Remove useless data
    df=df.loc[:,['GHI', 'DHI', 'BNI']]
    # fill empty area
    df=df.fillna(0)
    # Open meteorologic data
    met=pd.read_csv(met_file, header=22, sep=';')
    # Create date columns
    date=pd.date_range(start=start, end=end, freq='H', tz=cop.tz)
    #Remove useless data
    met=met.loc[:, ['Temperature', 'Pressure', 'Wind speed']]
    #Convert Kelvin to celcius 
    met['Temperature']=met['Temperature']-273.15
    #Merge two files 
    meteo=df
    meteo['wind_speed']=met['Wind speed'].values
    meteo['temp_air']=met['Temperature'].values
    meteo['pressure']=met['Pressure'].values
    #Change columns name
    meteo=meteo.rename(index=str, columns={'GHI':'ghi',
                                       'BNI': 'dni',
                                       'DHI':'dhi'
                                      })
    #Change index to date data
    meteo.set_index(pd.DatetimeIndex(date), inplace=True)
    
    # Define and choose modules and inverter 
    cec_module=pvlib.pvsystem.retrieve_sam(mod)
    module=cec_module[module]
    cec_inverter=pvlib.pvsystem.retrieve_sam(inver)
    inverter=cec_inverter[inverter]

    # Define  system
    system = pvsystem.PVSystem(surface_tilt=surface_tilt, 
                          surface_azimuth=surface_azimuth,
                          albedo=0.2,
                          modules_per_string=modules_per_string,
                          strings_per_inverter=strings_per_inverter,
                         module_parameters=module,
                          inverter_parameters=inverter,
                          name='test 1')

    mc=pvlib.modelchain.ModelChain(system,location,
                               aoi_model='physical',
                               transposition_model='perez',
                               dc_model='desoto',
                               ac_model='snlinverter',
                               spectral_model='no_loss',
                               losses_model='no_loss',
                               name=location.name)
    
    #run mode
    mc.run_model(date, meteo)
    # combine results into existing
    df_fixed=pd.concat([meteo,mc.total_irrad],axis=1)
    df_fixed=pd.concat([df_fixed,mc.temps],axis=1)
    df_fixed=pd.concat([df_fixed,mc.dc],axis=1)
    df_fixed['p_ac']=mc.ac
    return df_fixed

## Function to get 12 representative days
Use to run the code faster

In [3]:
def get12days(df, name='rpday.csv'):
    rpday=df.groupby([df.index.month, df.index.hour]).mean()
    rpday=rpday.loc[:,['ghi','p_ac']]
    rpday=rpday.to_csv(name)
    return rpday

## Function to get the best tilt
Use to have the best PV production throughout the year

In [4]:
def definetilt(min_tilt=0, maxtilt=90, step=1, filename='tiltsel.csv'):
    storemin=[]
    for surface_tilt in range(0,91,1):
        df=prod(location,irrad_file, met_file, start, end,mod, inver, module,inverter, surface_azimuth, surface_tilt,
            modules_per_string,strings_per_inverter )
        resample=df.resample('M').mean()
        resample['Month']=resample.index.month
        minv=resample.loc[resample['p_ac'].idxmin()]
        annual_mean=df['p_ac'].mean()
        storemin.append((surface_tilt, minv.Month, minv.p_ac, annual_mean))
    tilt=pd.DataFrame(storemin, columns=['surface_tilt', 'Month', 'p_ac', 'annual_mean'])
    best=tilt.loc[tilt['p_ac'].idxmax()]
    surface_tilt=best.surface_tilt
    tilt=tilt.to_csv(filename)
    return surface_tilt

## Set the data to use them in Oemof

**Don't forget to normalize your PV production. If not, Oemof won't work**

In [13]:
def file4oemof(df, filename='dataoemof.csv'):
    rpday=df.groupby([df.index.month, df.index.hour]).mean()
    #Here you used the pressure to generate a constant electriciy demand, change it for your own demand
    rpday=rpday.loc[:,['pressure','p_ac']]
    #Normalizing the PV production
    rpday['p_ac']=rpday['p_ac']/49500
    rpday['p_ac'][rpday['p_ac'] < 0] = 0
    oemofile=rpday.rename(index=str, columns={'p_ac':'pv', 'pressure':'demand_el'})
    date_time_index = pd.date_range(start='2018-01-01', periods=12*24,freq='H')
    oemofile['date']=date_time_index
    oemofile.reset_index(drop=True)
    oemofile.set_index('date',inplace=True)
    oemofile=oemofile.to_csv(filename)

# Data to run the function

In [14]:
location=pvlib.location.Location(44.888, 0.806, 'Europe/Paris',100, 'Copperton')
irrad_file='irradiation-2018.csv'
met_file='SoDa_MERRA2_2018.csv'
start='2018-01-01 00:00'
end='2018-12-31 23:00'
# mod = 'SandiaMod' or 'CECMod'
mod='CECMod'
# inver = 'cecinverter' or 'SandiaInverter'
inver='SandiaInverter'
module='Canadian_Solar_CS6K_275P'
inverter='Solectria_Renewables_LLC__PVI_50_kW_480__480V__480V__CEC_2018_'
modules_per_string=10
strings_per_inverter=1
surface_azimuth=180 
surface_tilt=20


## Running the fonction

In [15]:
df=prod(location,irrad_file, met_file, start, end,mod, inver, module,inverter, surface_azimuth, surface_tilt,
            modules_per_string,strings_per_inverter) 